# Notebook with Minimal example to run hposuite

In [ ]:
# It is assumed that the datadir is hposuite/data

from lib.benchmarks import BENCHMARKS
from lib.optimizers import OPTIMIZERS

print(BENCHMARKS)
print(OPTIMIZERS)

## Running Random Search on a Functional Benchmark (eg: ACKLEY)

In [2]:
func_benchmark = BENCHMARKS["Ex_Functional_Bench"]
rs = OPTIMIZERS["RandomSearch"]

In [ ]:
from hposuite import create_study

study = create_study(
    name="hposuite_demo_rs_func",
    output_dir="../../hposuite-output",
    optimizers=rs,
    benchmarks=func_benchmark,
    num_seeds=5,
    budget=100,
)

study.optimize(overwrite=True)

## Running a BlackBox Optimizer on a Synthetic Benchmark

SMAC provides the BlackBoxFacade which is used as an example of a BlackBox Optimizer. \
Multi-fidelity Hartmann 3D with the correlation "good" serves as an example of a Synethetic Multi-fidelity Benchmark.

In [2]:
bb_optimizer = OPTIMIZERS["Ex_Blackbox_Opt"]
mf_syn_benchmark = BENCHMARKS["mfh3_good"]

In [ ]:
from hposuite import create_study

study = create_study(
    name="hposuite_demo_bb_mfsyn",
    output_dir="../../hposuite-output",
    optimizers=bb_optimizer,
    benchmarks=mf_syn_benchmark,
    num_seeds=5,
    budget=20,
)

study.optimize(overwrite=True)

## Running a MultiFidelity Optimizer on a Surrogate Benchmark

We use a Multi-fidelity Optimizer like Hyperband from SMAC on the PD1 Cifar 100 Wide Resnet 2048 Benchmark which is a Surrogate Benchmark provided by the MF-Prior-Bench Package

In [3]:
sg_benchmark = BENCHMARKS["Ex_Surrogate_Bench"]
mf_optimizer = OPTIMIZERS["Ex_Multifidelity_Opt"]

In [ ]:
from hposuite import create_study

study = create_study(
    name="hposuite_demo_sg_mf",
    output_dir="../../hposuite-output",
    optimizers=mf_optimizer,
    benchmarks=sg_benchmark,
    num_seeds=5,
    budget=100,
)

study.optimize(overwrite=True)

## Running a Multi-objective Optimizer on a Benchmark with Multiple Objectives

Optuna's NSGA2 Sampler is used here as an example of a Multi-Objective Optimizer. \
PD1 Cifar 100 Wide Resnet 2048 from MF-Prior-Bench is the Benchmark used with "valid_error_rate" and "train_cost" as the objectives.

In [2]:
mo_optimizer = OPTIMIZERS["Ex_MO_Opt"]
mo_benchmark = BENCHMARKS["Ex_Surrogate_Bench"]

In [ ]:
from hposuite import create_study

study = create_study(
    name="hposuite_demo_sg_mf",
    output_dir="../../hposuite-output",
    optimizers=mo_optimizer,
    benchmarks=(
        mo_benchmark,
        {
            "objectives": ["valid_error_rate", "train_cost"]
        }
    ),
    num_seeds=5,
    budget=100,
)

study.optimize(overwrite=True)